In [28]:
import os
import numpy as np

task = "storycloze"
all_re = []
for pattern_id in [0]:
    patter_re = []
    for bs in [2, 4, 8]:
        for lr in ["1e-5", "2e-5", "3e-5"]:
            outdir = f"/n/fs/nlp-mengzhou/space3/out/pet/{task}/pattern{pattern_id}/shot32/bs{bs}_lr{lr}"
            file = os.path.join(outdir, "bert-base-uncased-log.txt")
            lines = open(file, "r").readlines()
            lines = [line for line in lines if "EVAL" in line ]
            try:
                res = [eval(line[line.index("EVAL"):].strip()[6:]) for line in lines]
            
                collected_re = []
                for seed in [3]:
                    eval_re = [re["fewshot_validation"] for re in res if re["seed"] == seed]
                    fewshot_eval_re = [re["validation"] for re in res if re["seed"] == seed]
                    index = np.argmax(eval_re)
                    best_re = fewshot_eval_re[index]
                    patter_re.append(best_re*100)
            except:
                print(outdir)
    all_re.append(patter_re)

In [25]:
for row in np.array(all_re):
    score = np.sort(row)[-3:]
    print(f"{round(np.mean(score), 2)} ({round(np.std(score), 1)})")

55.95 (0.3)


In [4]:
task = "hellaswag"
all_re = []
for pattern_id in [2]:
    patter_re = []
    for bs in [2, 4, 8]:
        for lr in ["1e-5", "2e-5", "3e-5"]:
            outdir = f"/n/fs/nlp-mengzhou/space3/out/pet/{task}/pattern{pattern_id}/shot32/bs{bs}_lr{lr}/roberta-large/p2-i0"
            file = os.path.join(outdir, "eval-log.txt")
            lines = open(file, "r").readlines()
            lines = [line for line in lines if "'acc'" in line ]
            try:
                re = eval(lines[-1].strip().split(" - ")[-1])["acc"]
                patter_re.append(re*100)
            except:
                print(outdir)
    all_re.append(patter_re)

/n/fs/nlp-mengzhou/space3/out/pet/hellaswag/pattern2/shot32/bs2_lr2e-5/roberta-large/p2-i0
/n/fs/nlp-mengzhou/space3/out/pet/hellaswag/pattern2/shot32/bs4_lr1e-5/roberta-large/p2-i0
/n/fs/nlp-mengzhou/space3/out/pet/hellaswag/pattern2/shot32/bs4_lr2e-5/roberta-large/p2-i0
/n/fs/nlp-mengzhou/space3/out/pet/hellaswag/pattern2/shot32/bs4_lr3e-5/roberta-large/p2-i0
/n/fs/nlp-mengzhou/space3/out/pet/hellaswag/pattern2/shot32/bs8_lr1e-5/roberta-large/p2-i0
/n/fs/nlp-mengzhou/space3/out/pet/hellaswag/pattern2/shot32/bs8_lr2e-5/roberta-large/p2-i0
/n/fs/nlp-mengzhou/space3/out/pet/hellaswag/pattern2/shot32/bs8_lr3e-5/roberta-large/p2-i0


In [20]:
for row in np.array(all_re):
    score = np.sort(row)[-3:]
    print(f"{round(np.mean(score), 2)} ({round(np.std(score), 1)})")

31.25 (0.0)


In [22]:
all_re

[[31.23879705238,
  29.396534554869547,
  28.599880501892056,
  31.228838876717784,
  30.551682931686912,
  29.376618203545114,
  31.288587930691097,
  30.90021907986457,
  29.61561441943836]]

In [35]:
np.std(final_re)

1.3266499161421599

In [6]:
import json
def load_jsonl(file):
    examples = []
    for line in open(file, "r"):
        example = json.loads(line.strip())
        examples.append(example)
    return examples

In [45]:
def build_record_dev():
    fewglue_record_train = load_jsonl("/n/fs/nlp-mengzhou/space3/fewglue/FewGLUE/ReCoRD/train.jsonl")
    record_train = load_jsonl("/n/fs/nlp-mengzhou/space3/data/downstream/SuperGLUE/ReCoRD/train.jsonl")
    record_train_text = [re["passage"]["text"] for re in record_train] 
    fewglue_record_train_text = [re["passage"]["text"] for re in fewglue_record_train] 
    for text in fewglue_record_train_text:
        while text in record_train_text:
            index = record_train_text.index(text)
            record_train.pop(index)
            record_train_text.pop(index)
    dev_examples = np.random.permutation(record_train)[:32]
    outfile = "/n/fs/nlp-mengzhou/space3/fewglue/FewGLUE/ReCoRD/fewshot_val.jsonl"
    with open(outfile, "w") as f:
        for example in dev_examples:
            f.write(f"{json.dumps(example)}\n")            


In [43]:
def build_copa_dev():
    fewglue_record_train = load_jsonl("/n/fs/nlp-mengzhou/space3/fewglue/FewGLUE/COPA/train.jsonl")
    record_train = load_jsonl("/n/fs/nlp-mengzhou/space3/data/downstream/SuperGLUE/COPA/train.jsonl")
    record_train_text = [re["premise"] for re in record_train] 
    fewglue_record_train_text = [re["premise"] for re in fewglue_record_train] 
    for text in fewglue_record_train_text:
        while text in record_train_text:
            index = record_train_text.index(text)
            record_train.pop(index)
            record_train_text.pop(index)
    dev_examples = np.random.permutation(record_train)[:32]
    outfile = "/n/fs/nlp-mengzhou/space3/fewglue/FewGLUE/COPA/fewshot_val.jsonl"
    with open(outfile, "w") as f:
        for example in dev_examples:
            f.write(f"{json.dumps(example)}\n")

In [52]:
def build_storycloze_train_dev():
    train = open("/n/fs/nlp-mengzhou/space3/data/downstream/storycloze/spring2016.val.tsv", "r").readlines()
    permutated_train_examples = np.random.permutation(train)
    train_examples = permutated_train_examples[:32]
    dev_examples = permutated_train_examples[32:64]
    outfile = "/n/fs/nlp-mengzhou/space3/data/downstream/storycloze/fewshot_train.jsonl"
    with open(outfile, "w") as f:
        for example in train_examples:
            f.write(f"{example}")
    outfile = "/n/fs/nlp-mengzhou/space3/data/downstream/storycloze/fewshot_val.jsonl"
    with open(outfile, "w") as f:
        for example in dev_examples:
            f.write(f"{example}")

In [67]:
def build_piqa_train_dev():
    train = load_jsonl("/n/fs/nlp-mengzhou/space3/data/downstream/piqa/train.jsonl")
    train_label = open("/n/fs/nlp-mengzhou/space3/data/downstream/piqa/train-labels.lst", "r").readlines()
    train_label = [int(i) for i in train_label]
    index = np.random.permutation(len(train))
    train_index = index[:32]
    valid_index = index[32:64]
    train_examples = [train[i] for i in train_index]
    dev_examples = [train[i] for i in valid_index]
    train_labels = [train_label[i] for i in train_index]
    dev_labels = [train_label[i] for i in valid_index]
    outfile = "/n/fs/nlp-mengzhou/space3/data/downstream/piqa/fewshot_train.jsonl"
    with open(outfile, "w") as f:
        for example, label in zip(train_examples, train_labels):
            example["label"] = label
            f.write(f"{json.dumps(example)}\n")
    outfile = "/n/fs/nlp-mengzhou/space3/data/downstream/piqa/fewshot_val.jsonl"
    with open(outfile, "w") as f:
        for example, label in zip(dev_examples, dev_labels):
            example["label"] = label
            f.write(f"{json.dumps(example)}\n")
    valid = load_jsonl("/n/fs/nlp-mengzhou/space3/data/downstream/piqa/valid.jsonl")
    valid_labels = open("/n/fs/nlp-mengzhou/space3/data/downstream/piqa/valid-labels.lst", "r").readlines()
    valid_labels = [int(i) for i in valid_labels]
    outfile = "/n/fs/nlp-mengzhou/space3/data/downstream/piqa/val.jsonl"
    with open(outfile, "w") as f:
        for example, label in zip(valid, valid_labels):
            example["label"] = label
            f.write(f"{json.dumps(example)}\n")

In [13]:
def build_hellaswag_train_dev():
    base_dir = "/n/fs/nlp-mengzhou/space3/data/downstream/hellaswag"
    train = load_jsonl(f"{base_dir}/hellaswag_train.jsonl")
    index = np.random.permutation(len(train))
    train_index = index[:32]
    valid_index = index[32:64]
    train_examples = [train[i] for i in train_index]
    dev_examples = [train[i] for i in valid_index]
    outfile = f"{base_dir}/fewshot/fewshot_train.jsonl"
    with open(outfile, "w") as f:
        for example in train_examples:
            f.write(f"{json.dumps(example)}\n")
    outfile = f"{base_dir}/fewshot/fewshot_val.jsonl"
    with open(outfile, "w") as f:
        for example in dev_examples:
            f.write(f"{json.dumps(example)}\n")
    

In [14]:
build_hellaswag_train_dev()

In [68]:
build_piqa_train_dev()

In [27]:
len(record_train_text)

65677

In [40]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained("roberta-base")

/n/fs/nlp-mengzhou/anaconda3/envs/pytorch17/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
